In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from eipy.deep_ei import EnsembleIntegration

2023-11-15 16:05:58.976201: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 16:05:58.976258: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 16:05:58.976293: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 16:05:58.983597: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [4]:
y_train = np.where(y_train != 1, 0, y_train)
y_test = np.where(y_test != 1, 0, y_test)

In [5]:
import pickle as pkl
'''with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "rb") as file:
    X_train_resized=pkl.load(file=file)
file.close()'''
with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "rb") as file:
    X_test_resized=pkl.load(file=file)
file.close()

In [6]:
sample_size = 1000
X_sample = X_test_resized[:sample_size]
y_sample = y_test[:sample_size]

In [7]:
y_sample.shape

(1000, 1)

In [8]:
'''from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}'''

"from sklearn.utils.class_weight import compute_class_weight\nclass_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)\nclass_weight_dict = {i: weight for i, weight in enumerate(class_weights)}"

In [9]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
res_model = ResNet50(weights='imagenet', include_top=False)
for layer in res_model.layers:
    layer.trainable = False
averaging_layer = layers.GlobalAveragePooling2D()(res_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(averaging_layer)

res_model = models.Model(inputs=res_model.input, outputs=output_layer)
res_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
'''from skimage.transform import resize

new_size = (224, 224)

X_train_resized = np.empty((X_train.shape[0], new_size[0], new_size[1], 3))
X_test_resized = np.empty((X_test.shape[0], new_size[0], new_size[1], 3))

for i in range(X_train.shape[0]):
    X_train_resized[i] = resize(X_train[i], new_size, preserve_range=True, mode='reflect')
for i in range(X_test.shape[0]):
    X_test_resized[i] = resize(X_test[i], new_size, preserve_range=True, mode='reflect')
'''

"from skimage.transform import resize\n\nnew_size = (224, 224)\n\nX_train_resized = np.empty((X_train.shape[0], new_size[0], new_size[1], 3))\nX_test_resized = np.empty((X_test.shape[0], new_size[0], new_size[1], 3))\n\nfor i in range(X_train.shape[0]):\n    X_train_resized[i] = resize(X_train[i], new_size, preserve_range=True, mode='reflect')\nfor i in range(X_test.shape[0]):\n    X_test_resized[i] = resize(X_test[i], new_size, preserve_range=True, mode='reflect')\n"

In [11]:
'''with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "wb") as file:
    pkl.dump(X_train_resized, file=file)
file.close()
with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "wb") as file:
    pkl.dump(X_test_resized, file=file)
file.close()'''

'with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "wb") as file:\n    pkl.dump(X_train_resized, file=file)\nfile.close()\nwith open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "wb") as file:\n    pkl.dump(X_test_resized, file=file)\nfile.close()'

In [12]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNet(weights='imagenet', include_top=False)

x = base_model.output
for layer in base_model.layers:
    layer.trainable = False
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
net_model = Model(inputs=base_model.input, outputs=predictions)

net_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
base_predictors = {
    "ResNet50" : res_model,
    "MobileNet" : net_model
}

In [14]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=2,
                        k_inner=2,
                        n_samples=1,
                        sampling_aggregation=None,
                        sampling_strategy=None,
                        n_jobs=-1,# -1 could be messing up for one bp
                        metrics=metrics,
                        random_state=38,
                        project_name="images",
                        model_building=True,
                        )

In [15]:
EI.fit_base(X_sample, y_sample, modality_name="shmimage")

Training base predictors on shmimage...
        
... for ensemble performance analysis...


Generating ensemble training data: |          |  0%

INFO:tensorflow:Assets written to: ram://a00761079cf14856959528088493d1f2/assets


INFO:tensorflow:Assets written to: ram://a00761079cf14856959528088493d1f2/assets


INFO:tensorflow:Assets written to: ram://99f9d2a65bbd422e8239f6db04116185/assets


...

11/32 [=========>....................] - ETA: 7s - loss: 0.5224 - accuracy: 0.7642INFO:tensorflow:Assets written to: ram://7cb012627445446b823e1dedf896d46f/assets


INFO:tensorflow:Assets written to: ram://7cb012627445446b823e1dedf896d46f/assets


32/32 [==============================] - 12s 348ms/step - loss: 0.4087 - accuracy: 0.8570
INFO:tensorflow:Assets written to: ram://3b9a60d91337421eb1d78ee7a16b4087/assets


INFO:tensorflow:Assets written to: ram://3b9a60d91337421eb1d78ee7a16b4087/assets
2023-11-15 16:12:27.097184: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://144630fe4454467babe5d3564c3b444d: INVALID_ARGUMENT: ram://144630fe4454467babe5d3564c3b444d is a directory.
2023-11-15 16:12:29.224631: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://583a4b37a9ef488090d7a8b03d7b0104: INVALID_ARGUMENT: ram://583a4b37a9ef488090d7a8b03d7b0104 is a directory.


32/32 [==============================] - 4s 91ms/step - loss: 0.6437 - accuracy: 0.8710


In [16]:
EI.base_summary

{'metrics': modality        shmimage          
 base predictor MobileNet  ResNet50
 f_max           0.271845  0.357977
 auc             0.689673  0.750651
 mcc             0.122666  0.000000,
 'thresholds': modality        shmimage          
 base predictor MobileNet  ResNet50
 f_max           0.732705  0.025145
 auc                  NaN       NaN
 mcc             0.500000  0.500000}

In [17]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
ensemble_predictors = {"LR": LogisticRegression(),
                       "XGB": XGBClassifier()}
EI.fit_ensemble(ensemble_predictors=ensemble_predictors)

Analyzing ensembles: |          |  0%

Analyzing ensembles: |██████████|100%
Training final ensemble models: |██████████|100%


In [18]:
EI.ensemble_summary

{'metrics':              LR       XGB
 f_max  0.314961  0.187500
 auc    0.705583  0.505414
 mcc    0.000000  0.008863,
 'thresholds':              LR       XGB
 f_max  0.089109  0.085131
 auc         NaN       NaN
 mcc    0.500000  0.500000}

In [19]:
X_dict = {"shmimage": X_sample}

In [20]:
EI.modality_names

['shmimage']

In [21]:
EI.predict(X_dict=X_dict, ensemble_model_key="LR")

2023-11-15 16:12:44.751956: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://a0c372635c3a4a9fafd501fe4374652c: INVALID_ARGUMENT: ram://a0c372635c3a4a9fafd501fe4374652c is a directory.


32/32 [==============================] - 12s 340ms/step


2023-11-15 16:12:58.753628: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://43eb3e2192ab48a48172014196bcb629: INVALID_ARGUMENT: ram://43eb3e2192ab48a48172014196bcb629 is a directory.


32/32 [==============================] - 3s 85ms/step


array([0.0799678 , 0.13168917, 0.08950173, 0.11083812, 0.0693496 ,
       0.06736746, 0.10310862, 0.06858966, 0.07082725, 0.09452216,
       0.07259091, 0.12944848, 0.07248769, 0.0842082 , 0.11139511,
       0.0849701 , 0.0869451 , 0.08038993, 0.09722135, 0.0635078 ,
       0.08000077, 0.06666598, 0.06523644, 0.19327766, 0.06848172,
       0.08309012, 0.07751799, 0.0761429 , 0.13700303, 0.06413451,
       0.06764469, 0.07047187, 0.08831956, 0.08185731, 0.12216659,
       0.07493386, 0.06820865, 0.10972586, 0.14345645, 0.08331853,
       0.11933513, 0.06925324, 0.08742322, 0.0691075 , 0.1129003 ,
       0.15530523, 0.06576854, 0.10803305, 0.06390492, 0.07220081,
       0.08586769, 0.07413792, 0.06842103, 0.07127218, 0.10611342,
       0.08890289, 0.07251464, 0.07270016, 0.06875327, 0.07691401,
       0.06553054, 0.06343942, 0.09463463, 0.08855341, 0.08109449,
       0.08100689, 0.16081715, 0.0722621 , 0.08212006, 0.07890292,
       0.08349745, 0.06911743, 0.07547345, 0.12128167, 0.10148